In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# initialize seed, so we get reproducible results
from numpy.random import seed
import tensorflow as tf
import os
import random
from tensorflow.python.eager import context

SEED_VALUE = 0
os.environ["PYTHONHASHSEED"]=str(SEED_VALUE)
random.seed(SEED_VALUE)

seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
# import the rare dataset
rare_ds = pd.read_csv("/kaggle/input/rare-kubernetes-anomalies/RARE.csv", sep="|", header=0, low_memory=False)
rare_ds.head()
original_rare_ds = rare_ds.copy()

In [4]:
print(rare_ds.shape)

In [5]:
rare_ds.dtypes.unique()

We can observe that we have a time series dataset, which contains the timestamp as first column, and anomaly as second column. Thee is the isntance field, that contains a descriptive value of the cluster status on that moment.
Then, there are a set of prometheus metrics, collected for the specific timestamp. The anomaly is the field that we are going to use for training, and the other entries are numerical entries, that are going to influence the model.
We have a total of 10.010 rows for training, and a total of 7063 features, that will need to be selected.
Let's show the different values for the anomaly and instance field:

In [6]:
rare_ds.columns =[col.strip() for col in rare_ds.columns]
print(rare_ds["anomaly"].unique())
print(rare_ds["instance"].unique())

We can see that the anomaly is just a binary field, that we can use to know if there is an anomaly or not. We can check the distribution of the field. 

In [7]:
rare_ds.hist(column='anomaly')

We can see that the anomaly is just a binary field, that we can use to know if there is an anomaly or not. For our case, we can drop the instance field, as it is not relevant for training based on anomalies.

In [8]:
rare_ds.drop(labels=['instance'], axis=1, inplace = True)


 Now we need to proceed with feature selection. We have a total of 7060 metrics and the objective is to select the relevant ones for the project. In order to do it, we could proceed with manual removal first, using the Kubernetes domain knowledge, to discard the non meaningful ones:
 - any information about resource creation is not relevant, because it depends on the workloads that are existing on each cluster
 - kafka topics, logs... are a consequence of other operations in the cluster, they cannot be part of cluster failures reason
 - any information related about pod status, container status.. that have some fixed id needs to be discarded, as it is something that will change depending on the workloads and cluster

In [9]:
final_columns = []
for column in sorted(rare_ds.columns):
    if not column.strip().startswith(("kafka_topic", "kafka_log", "kube_configmap", "kube_namespace", "kube_service", "kube_secret", "kube_pod_status", "kube_pod_container_status", "scrape_samples", "kafka_server_brokertopic")):
        final_columns.append(column.strip())

In [10]:
features_ds = rare_ds[final_columns].copy()
features_ds.shape

After manual selection we will use automated methods. First step would be to remove the constant features. Any metric that is having a constant value, with no relevant variance over the dataset, is a metric that is not useful to discriminate a target. We can find those constant values using variance:

In [11]:
features_ds.dtypes.unique()

In [12]:
# for all columns that are objects, convert to float
for column in features_ds.columns:
    if features_ds[column].dtype == "object":
        features_ds[column] = features_ds[column].astype(float)
features_ds.dtypes.unique()

In [13]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0.1)
sel.fit(features_ds)

In [14]:
# get a list of the constant features. Those can be removed
print(
    len([
        x for x in features_ds.columns
        if x not in features_ds.columns[sel.get_support()]
    ]))

In [15]:
to_strip = [x for x in features_ds.columns if x not in features_ds.columns[sel.get_support()]]
features_ds_strip = features_ds.drop(labels=to_strip, axis=1)
features_ds_strip.shape

With that technique we have reduced the number of features to 1740. It is still a long number of features, so we need to apply more feature reduction techniques to select the most relevant ones. As we deal with a large amount of features, we will reduce them using Mutual Information technique:

In [16]:
X = features_ds_strip
Y = rare_ds["anomaly"].to_frame()
print(X.shape)
print(Y.shape)

In [17]:
# select with mutual information
from sklearn.feature_selection import SelectPercentile as SP
from sklearn.model_selection import train_test_split as tts
from sklearn.tree import DecisionTreeClassifier as DTC

selector = SP(percentile=2) # select features with top 2% MI scores
selector.fit(X,Y.values.ravel())
X_4 = selector.transform(X)
X_train_4,X_test_4,y_train,y_test = tts(
    X_4,Y
    ,random_state=0
    ,stratify=Y
)
model_4 = DTC().fit(X_train_4,y_train)
score_4 = model_4.score(X_test_4,y_test)

In [18]:
print(f"score_4:{score_4}")
print(X_4.shape)

In [19]:
# get columns, we have reduced to a total of 35
columns = np.asarray(X.columns.values)
support = np.asarray(selector.get_support())
columns_with_support = columns[support]
print(columns_with_support)

Using the Mutual Information method, for the 3% with bigger MI score, returned a total of 36 columns. With a more reasonable number of features, we can now use visualization to discard the highly correlated ones.

In [20]:
# with this reduced amount of features, now we can do a correlation map
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt
x = rare_ds[columns_with_support]
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [21]:
# retrieve the top correlated values to remove them. We are going to list the pairs with more correlation
# and remove vars over that threshold, so we reduce the redundant features
c = x.corr().abs().unstack().sort_values().drop_duplicates()
c = c[c>0.9]

to_remove = ["java_lang_garbagecollector_committed_lastgcinfo_memoryusageaftergc_g1_young_generation_key_g1_eden_space_0", 
             "java_lang_garbagecollector_committed_lastgcinfo_memoryusagebeforegc_g1_young_generation_key_g1_old_gen_0",
             "java_lang_memorypool_committed_collectionusage_g1_old_gen_0", "node_memory_Cached_bytes_0",
            "node_memory_Inactive_file_bytes_0", "node_memory_AnonPages_bytes_0", "node_memory_Active_anon_bytes_0",
             "node_memory_Slab_bytes_0", "java_lang_operatingsystem_systemloadaverage_0", "node_memory_Inactive_bytes_0",
             "node_memory_Active_file_bytes_0", "node_memory_SUnreclaim_bytes_0", "java_lang_memorypool_committed_usage_g1_old_gen_0",
             "java_lang_garbagecollector_committed_lastgcinfo_memoryusageaftergc_g1_young_generation_key_g1_old_gen_0"
             ]
    
columns_to_remove = list(set(to_remove))
common_features = set(x.columns) - set(columns_to_remove)
print(common_features)


Once the common features have been selected, let's do some basic analysis on them, and will plot them to show their trend and seasonality. We can see that each var has independent distribution.

In [22]:
rare_ds[common_features].describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [23]:
rare_ds[common_features].plot(subplots=True, layout=(4,6), figsize=(25,25))

The data inspection shows that the variables have different value ranges, so we will need to scale them in order to apply training models.

In [24]:
from sklearn.preprocessing import MinMaxScaler
final_df = rare_ds[common_features]
final_df.head()

In [25]:
# add the anomaly column to the final dataframe to compute it
final_df = pd.concat([rare_ds[['anomaly']], final_df], axis=1)

Next thing will be to convert the timestamp to datetime, and then we need to prepare data to have regular intervals. We check the interval time, and we can see that we have information for over 15 hours, for one day. We can see that the data is already divided into regular intervals of one second. The next step is to tranform this data into a time series problem. In orer to do that, first we need to decide the time window. This is where the data we dropped before - with Instance information - can be useful, because it shows the time where the failure was introduced, and the time it took to fail. Let's plot it: 

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})
original_rare_ds.columns =[col.strip() for col in original_rare_ds.columns]
original_df = original_rare_ds.set_index("time")
original_df['anomaly'].plot(linewidth=0.5);

In [27]:
original_df[['anomaly']].iloc[0:1000].plot(linewidth=0.5);

In [28]:
original_df[['anomaly']].iloc[3000:3500].plot(linewidth=0.5);

We can see that the tests have been executed regularly in intervals of over 400 iterations. The anomaly is present in an interval around 25 iterations:

In [29]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [30]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# integer encode direction
encoder = LabelEncoder()
values=final_df.values
values[:,1] = encoder.fit_transform(values[:,1])

In [31]:
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [32]:
PERIOD=300

# first drop the features for last period
reframed = series_to_supervised(scaled, PERIOD, PERIOD)
reframed.drop(reframed.columns[list(range(len(reframed.columns)-len(common_features),len(reframed.columns)))], axis=1, inplace=True)

# and also drop the var1 (variable to be predicted, but first extracting the last one)
target_var = reframed.iloc[:,-1:]
reframed = reframed[reframed.columns.drop(list(reframed.filter(regex='var1')))]

reframed.head()

In [33]:
target_var.head()

In [34]:
print(reframed.shape)

In [35]:
# split into train and test sets
values = reframed.values
target_values = target_var.values.astype(int)

train_time=int(values.shape[0]*0.8)
test_time=int(values.shape[0]*0.9)

train_X = values[:train_time, :]
valid_X = values[train_time:test_time, :]
test_X = values[test_time:, :]

train_y = target_values[:train_time, :]
valid_y = target_values[train_time:test_time, :]
test_y = target_values[test_time:, :]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape, test_y.shape)

To train our model, we are interested in the F1 score, not accuracy. The reason is that we want to predict properly positive and negative classes, and as the dataset is imbalanced, accuracy will be a false metric, because it will give high results even if it fails in the positive class. So, we can create a custom f1 score metric, and train based on it.

In [36]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

Once we have the data in the proper shape, and it is divided properly into train/validation/test, is time to start applying a model. We are going to use an LSTM model, that will be able to pick information from a sequence and will be able to learn patterns, specially if they are long sequences. 

In [37]:
### design network
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.metrics import AUC

model1 = Sequential()
model1.add(LSTM(units=50, return_sequences=False, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
model1.add(Dense(units=10, activation='relu'))
model1.add(Dense(units=1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC(name='auc'), AUC(name='prc', curve='PR')])
model1.summary()

In [38]:
# fit network
from keras.callbacks import EarlyStopping

EPOCHS=100
es = EarlyStopping(monitor=get_f1, mode='max', verbose=0, patience=10, restore_best_weights=True)

history1 = model1.fit(train_X, train_y, epochs=EPOCHS, batch_size=60, validation_data=(valid_X, valid_y), verbose=0, shuffle=False, callbacks=[es])

In [39]:
from matplotlib import pyplot
pyplot.plot(history1.history['loss'], label='train')
pyplot.plot(history1.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [40]:
pyplot.plot(history1.history['prc'], label='train')
pyplot.plot(history1.history['val_prc'], label='test')
pyplot.legend()
pyplot.show()

In [41]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def evaluate_and_predict(model, test_X, test_y):
    scores = model.evaluate(test_X, test_y, verbose=0)
    print("AUC ROC: {}".format(scores[1]))
    print("AUC precision-recall: {}".format(scores[2]))  
    
    y_pred = model.predict(test_X)
    predict_class = ((y_pred > 0.5)+0).ravel()
    print(confusion_matrix(test_y, predict_class))
    print("F1 score for 50%: {}".format(f1_score(test_y, predict_class)))
    
    predict_class = ((y_pred > 0.2)+0).ravel()
    print(confusion_matrix(test_y, predict_class))
    print("F1 score for 20%: {}".format(f1_score(test_y, predict_class)))

We have plotted the chart to see the train and validation loss, and PR curve. We can see that model is finally converging.

Now let's predict values with the testing data we separated from our dataset.

In [42]:
evaluate_and_predict(model1, test_X, test_y)

Although the ROC and PR metrics are quite good, the F1 score and the ability to predict classes correctly - specially the positive class - is still not accurate. So we will try different options.

In [43]:
history2 = model1.fit(train_X, train_y, epochs=EPOCHS, batch_size=PERIOD, validation_data=(valid_X, valid_y), verbose=0, shuffle=False, callbacks=[es])

In [44]:
pyplot.plot(history2.history['loss'], label='train')
pyplot.plot(history2.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [45]:
pyplot.plot(history2.history['prc'], label='train')
pyplot.plot(history2.history['val_prc'], label='test')
pyplot.legend()
pyplot.show()

In [46]:
evaluate_and_predict(model1, test_X, test_y)

In [47]:
model3 = Sequential()
model3.add(LSTM(units=256, return_sequences=False, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
model3.add(Dense(units=10, activation='relu'))
model3.add(Dense(units=1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC(name='auc'), AUC(name='prc', curve='PR')])
model3.summary()
history3 = model3.fit(train_X, train_y, epochs=EPOCHS, batch_size=PERIOD, validation_data=(valid_X, valid_y), verbose=0, shuffle=False, callbacks=[es])

In [48]:
pyplot.plot(history3.history['loss'], label='train')
pyplot.plot(history3.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [49]:
pyplot.plot(history3.history['prc'], label='train')
pyplot.plot(history3.history['val_prc'], label='test')
pyplot.legend()
pyplot.show()

We can see good results, where validation loss is finally converging, with some regular intervals. Let's evaluate the model to decide which one is best:

In [50]:
evaluate_and_predict(model3, test_X, test_y)

In [51]:
model4 = Sequential()
model4.add(LSTM(units=512, return_sequences=False, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
model4.add(Dense(units=256, activation='relu'))
model4.add(Dropout(0.2, seed=SEED_VALUE))
model4.add(Dense(units=1, activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC(name='auc'), AUC(name='prc', curve='PR')])
model4.summary()

In [52]:
history4 = model4.fit(train_X, train_y, epochs=EPOCHS, batch_size=PERIOD, validation_data=(valid_X, valid_y), verbose=0, shuffle=False, callbacks=[es])

In [53]:
pyplot.plot(history4.history['loss'], label='train')
pyplot.plot(history4.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [54]:
pyplot.plot(history4.history['prc'], label='train')
pyplot.plot(history4.history['val_prc'], label='test')
pyplot.legend()
pyplot.show()

In [55]:
evaluate_and_predict(model4, test_X, test_y)

We can see that the model has wors precision/recall than previews, and also worse F1 score, but is predicting positive classes in a more accurate way. So if our intention is to predict positive classes, we should pick this one.